In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd

def carregar_dados(caminho_arquivo):
    dados = pd.read_csv(caminho_arquivo)
    dados['Date'] = pd.to_datetime(dados['Date'])
    dados.set_index('Date', inplace=True)
    return dados



In [ ]:
caminhos = {
    'META': {
        '3m': '/content/drive/My Drive/trabalhos/trabalho_bi/META.csv',
        '1y': '/content/drive/My Drive/trabalhos/trabalho_bi/META (1).csv',
        '5y': '/content/drive/My Drive/trabalhos/trabalho_bi/META (2).csv'
    },
    'AAPL': {
        '3m': '/content/drive/My Drive/trabalhos/trabalho_bi/AAPL.csv',
        '1y': '/content/drive/My Drive/trabalhos/trabalho_bi/AAPL (1).csv',
        '5y': '/content/drive/My Drive/trabalhos/trabalho_bi/AAPL (2).csv'
    },
    'BA': {
        '3m': '/content/drive/My Drive/trabalhos/trabalho_bi/BA.csv',
        '1y': '/content/drive/My Drive/trabalhos/trabalho_bi/BA (1).csv',
        '5y': '/content/drive/My Drive/trabalhos/trabalho_bi/BA (2).csv'
    },
    'JNJ': {
        '3m': '/content/drive/My Drive/trabalhos/trabalho_bi/JNJ.csv',
        '1y': '/content/drive/My Drive/trabalhos/trabalho_bi/JNJ (1).csv',
        '5y': '/content/drive/My Drive/trabalhos/trabalho_bi/JNJ (2).csv'
    },
    'XOM': {
        '3m': '/content/drive/My Drive/trabalhos/trabalho_bi/XOM.csv',
        '1y': '/content/drive/My Drive/trabalhos/trabalho_bi/XOM (1).csv',
        '5y': '/content/drive/My Drive/trabalhos/trabalho_bi/XOM (2).csv'
    }
}


In [ ]:
dados_empresas = {empresa: {periodo: carregar_dados(caminhos[empresa][periodo]) for periodo in caminhos[empresa]} for empresa in caminhos}



In [ ]:
def acessar_dado(empresa, periodo):
    return dados_empresas.get(empresa, {}).get(periodo, None)


In [ ]:
import matplotlib.pyplot as plt

def plotar_dados(dados, titulo):
    plt.figure(figsize=(12, 6))
    plt.plot(dados['Close'], label='Preço de Fechamento')
    plt.title(titulo)
    plt.xlabel('Data')
    plt.ylabel('Preço de Fechamento')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
def plotar_medias_moveis(dados, titulo):
    dados['MA30'] = dados['Close'].rolling(window=30).mean()
    dados['MA90'] = dados['Close'].rolling(window=90).mean()

    plt.figure(figsize=(12, 6))
    plt.plot(dados['Close'], label='Preço de Fechamento')
    plt.plot(dados['MA30'], label='Média Móvel 30 dias', color='orange')
    plt.plot(dados['MA90'], label='Média Móvel 90 dias', color='red')
    plt.title(titulo)
    plt.xlabel('Data')
    plt.ylabel('Preço de Fechamento')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
import matplotlib.pyplot as plt

def salvar_grafico(dados_empresas, periodo, nome_arquivo):
    plt.figure(figsize=(12, 8))

    for empresa in dados_empresas:
        dados = acessar_dado(empresa, periodo)
        if dados is not None:
            plt.plot(dados.index, dados['Close'], label=empresa)

    plt.title(f'Preço de Fechamento - Período: {periodo}', fontsize=16)
    plt.xlabel('Data', fontsize=14)
    plt.ylabel('Preço de Fechamento', fontsize=14)
    plt.legend(loc='upper left', fontsize=12)
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(nome_arquivo)
    plt.close()

periodos = {
    '3m': '/content/drive/My Drive/trabalhos/trabalho_bi/grafico_3m.png',
    '1y': '/content/drive/My Drive/trabalhos/trabalho_bi/grafico_1y.png',
    '5y': '/content/drive/My Drive/trabalhos/trabalho_bi/grafico_5y.png'
}

for periodo, arquivo in periodos.items():
    salvar_grafico(dados_empresas, periodo, arquivo)


In [ ]:



def calcular_valor_investimento(dados, investimento_inicial=1000):
    dados = dados.copy()
    dados['Retorno Diário'] = dados['Close'].pct_change()
    dados['Valor Investimento'] = investimento_inicial * (1 + dados['Retorno Diário']).cumprod()
    return dados

def plotar_valor_investimento(dados_empresas, periodo):
    fig, ax = plt.subplots(figsize=(12, 8))

    for empresa in dados_empresas:
        dados = acessar_dado(empresa, periodo)
        if dados is not None:
            dados_valor = calcular_valor_investimento(dados)
            ax.plot(dados_valor.index, dados_valor['Valor Investimento'], label=empresa)

    ax.set_title(f'Valor do Investimento - Período: {periodo}', fontsize=16)
    ax.set_xlabel('Data', fontsize=14)
    ax.set_ylabel('Valor do Investimento', fontsize=14)
    ax.legend(loc='upper left', fontsize=12)
    ax.grid(True)


    caminho_imagem = f'/content/drive/My Drive/valor_investimento_{periodo}.png'
    plt.savefig(caminho_imagem)
    plt.close()


periodos = ['3m', '1y', '5y']

for periodo in periodos:
    plotar_valor_investimento(dados_empresas, periodo)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


def calcular_volatilidade(dados):
    dados = dados.copy()
    dados['Retorno Diário'] = dados['Close'].pct_change()
    dados['Volatilidade'] = dados['Retorno Diário'].rolling(window=21).std()  # 21 dias úteis em um mês
    return dados


def plotar_volatilidade(dados_empresas, periodo):
    fig, ax = plt.subplots(figsize=(12, 8))

    for empresa in dados_empresas:
        dados = acessar_dado(empresa, periodo)
        if dados is not None:
            dados_volatilidade = calcular_volatilidade(dados)
            ax.plot(dados_volatilidade.index, dados_volatilidade['Volatilidade'], label=empresa)

    ax.set_title(f'Volatilidade - Período: {periodo}', fontsize=16)
    ax.set_xlabel('Data', fontsize=14)
    ax.set_ylabel('Volatilidade', fontsize=14)
    ax.legend(loc='upper left', fontsize=12)
    ax.grid(True)

    plt.savefig(f'volatilidade_{periodo}.png')
    plt.close()

periodos = ['3m', '1y', '5y']


for periodo in periodos:
    plotar_volatilidade(dados_empresas, periodo)


In [ ]:

def calcular_volatilidade(dados):
    dados = dados.copy()
    dados['Retorno Diário'] = dados['Close'].pct_change()
    dados['Volatilidade'] = dados['Retorno Diário'].rolling(window=21).std()
    return dados

def plotar_volatilidade(dados_empresas, periodo):
    fig, ax = plt.subplots(figsize=(12, 8))

    for empresa in dados_empresas:
        dados = acessar_dado(empresa, periodo)
        if dados is not None:
            dados_volatilidade = calcular_volatilidade(dados)
            ax.plot(dados_volatilidade.index, dados_volatilidade['Volatilidade'], label=empresa)

    ax.set_title(f'Volatilidade - Período: {periodo}', fontsize=16)
    ax.set_xlabel('Data', fontsize=14)
    ax.set_ylabel('Volatilidade', fontsize=14)
    ax.legend(loc='upper left', fontsize=12)
    ax.grid(True)

    caminho_imagem = f'/content/drive/My Drive/volatilidade_{periodo}.png'
    plt.savefig(caminho_imagem)
    plt.close()

periodos = ['3m', '1y', '5y']

for periodo in periodos:
    plotar_volatilidade(dados_empresas, periodo)

In [ ]:

def calcular_correlacao(dados_empresas, periodo):
    retornos = {}

    for empresa in dados_empresas:
        dados = acessar_dado(empresa, periodo)
        if dados is not None:
            dados['Retorno Diário'] = dados['Close'].pct_change()
            retornos[empresa] = dados['Retorno Diário']

    retornos_df = pd.DataFrame(retornos)
    correlacao = retornos_df.corr()

    return correlacao

def plotar_correlacao(dados_empresas, periodo):
    correlacao = calcular_correlacao(dados_empresas, periodo)

    plt.figure(figsize=(10, 8))
    cax = plt.matshow(correlacao, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar(cax)

    plt.title(f'Correlação entre Ativos - Período: {periodo}', fontsize=16)
    plt.xticks(range(len(dados_empresas)), dados_empresas.keys(), rotation=45)
    plt.yticks(range(len(dados_empresas)), dados_empresas.keys())

    caminho_imagem = f'/content/drive/My Drive/correlacao_{periodo}.png'
    plt.savefig(caminho_imagem)
    plt.close()

periodos = ['3m', '1y', '5y']

for periodo in periodos:
    plotar_correlacao(dados_empresas, periodo)

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>